In [4]:
import json

# 读取 JSON 文件并解析为 Python 对象
def count_elements_in_json(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return len(data)

# 指定 JS/home/qiang/projects/Digital_mental_health/Dataset/Condidate_sentiment_datasetON 文件路径
json_file = "/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/IAS2EAS/IAS2EAS_event/IAS_EAS_800_event_extraction_clustered_deduplication.json"

# 统计元素数量
element_count = count_elements_in_json(json_file)
print(f"JSON 文件中包含 {element_count} 个元素。")

JSON 文件中包含 714 个元素。


prompt_few_shot="An event is defined as any stimulus that an individual\'s \
                environment or within that individual(e.g. thoughts of feelings) that has a good or bad effect from the individual\'s \
                point of view. Events can be mental(e.g. I was afraid), social(e.g. I got a pay raise) or physical(e.g. I got in a car accident).\
                Events should be unambiguously good or bad from the individual\'s point of view and may occur in the past, present or hypothetical future.  \
                Please analyze and determine the provided events. We should retain events that are unfavorable for the participants, i.e., bad events. \
                These events should not state facts and one's emotion or pose questions but rather describe a trigger for a bad situation for the participants. \
                Please keep such events and rewrite them into concise and fluent versions. For events that do not meet the requirements, please output:None\
                If there is no event in the post, you should output:\"Event:None\"\
                Please refer to the examples I gave:\
                <example>\
                Event: I snore REALLY loudly and sound like I'm not breathing, which means I must be completely unhealthy and will probably die young from some obscure disease, and no one will ever want to sleep next to me again.\
                Event: I snore really loudly and sound like I'm not breathing.\
                </example>\
                <example>\
                Event: I'm unable to work due to worsening disabilities and severe health issues, which means my whole life is basically over - no career success, no financial stability, no social status, and no hope for a better future.\
                Event: I'm unable to work due to worsening disabilities and severe health issues.\
                </example>\
                <example>\
                Event: why are you sad?\
                Event: None\
                </example>\
                <example>\
                Event: I bet you received lots of hit from that tweet; at work i cannot, wish i could\
                Event: I received hit from that tweet\
                </example>\
                <example>\
                Event: I`m so sad, really really sad\
                Event: None\
                </example>\
                You should answer with a specific format. For example, you should output:\"Event:...\"\
                Please understand what is event that we need from above explanation and extract the main event from the following paragraph:"
          


In [2]:
from langchain.llms import Ollama
import json
import os
import pandas as pd
from tqdm import tqdm
llm = Ollama(model="llama3:70b")


prompt_few_shot="An event is defined as any stimulus that an individual\'s \
                environment or within that individual(e.g. thoughts of feelings) that has a good or bad effect from the individual\'s \
                point of view. Events can be mental(e.g. I was afraid), social(e.g. I got a pay raise) or physical(e.g. I got in a car accident).\
                Events should be unambiguously good or bad from the individual\'s point of view and may occur in the past, present or hypothetical future.  \
                Please analyze and determine the provided events. We should retain events that are unfavorable for the participants, i.e., bad events. \
                These events should not state facts and one's emotion or pose questions but rather describe a trigger for a bad situation for the participants. \
                Please keep such events and rewrite them into concise and fluent versions. For events that do not meet the requirements, please output:None\
                If there is no event in the post, you should output:\"Event:None\"\
                Please refer to the examples I gave:\
                <example>\
                Event: I snore REALLY loudly and sound like I'm not breathing, which means I must be completely unhealthy and will probably die young from some obscure disease, and no one will ever want to sleep next to me again.\
                Event: I snore really loudly and sound like I'm not breathing.\
                </example>\
                <example>\
                Event: I'm unable to work due to worsening disabilities and severe health issues, which means my whole life is basically over - no career success, no financial stability, no social status, and no hope for a better future.\
                Event: I'm unable to work due to worsening disabilities and severe health issues.\
                </example>\
                <example>\
                Event: why are you sad?\
                Event: None\
                </example>\
                <example>\
                Event: I bet you received lots of hit from that tweet; at work i cannot, wish i could\
                Event: I received hit from that tweet\
                </example>\
                <example>\
                Event: I`m so sad, really really sad\
                Event: None\
                </example>\
                You should answer with a specific format. For example, you should output:\"Event:...\"\
                Please understand what is event that we need from above explanation and extract the main event from the following paragraph:"
          

def event_extraction(input_file, output_folder,posts):
    data_list = []

    for data in tqdm(posts, desc="Processing Event"):
        try:
            prompt=prompt_few_shot+str(data)
        except:
            print("some errors")
        res = llm.predict(prompt)
        result_str = res.split("Event:", 1)[-1].strip()
        data_res_dict = {
            "Post": data,
            "Event": result_str
        }
        data_list.append(data_res_dict)

    output_json_path = os.path.join(output_folder, os.path.basename(input_file).split('.')[0]+'_event_extraction.json')


    with open(output_json_path, 'w') as json_file:
        json.dump(data_list, json_file, indent=4)


def extract_text_in_json(json_file):
    with open(json_file, 'r') as file:
        try:
            data = json.load(file)
        except:
            print("error")
        Post = [o["Post"] for o in data]
        # print(events)
        return Post

def extract_text_column(csv_file_path):
    # 加载CSV文件
    data = pd.read_csv(csv_file_path, encoding='latin1')
    
    # 检查"text"列是否存在于DataFrame中
    if 'text' in data.columns:
        # 提取"text"列
        text_data = data['text']
        return text_data
    else:
        raise ValueError("The CSV file does not contain a 'text' column.")

# 指定CSV文件的路径
csv_file_path = '/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/GAS_SPAS_EVENT.json'
output_folder='/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/'
# 调用函数并打印结果
try:
    text_column = extract_text_in_json(csv_file_path)
    event_extraction(csv_file_path, output_folder, text_column)
except Exception as e:
    print(e)

print("处理完成。")


Processing Event: 100%|██████████| 1852/1852 [43:00<00:00,  1.39s/it]

处理完成。


In [5]:
import json
from sentence_transformers import SentenceTransformer, util

# 加载JSON文件
with open('/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/GAS_SPAS_EVENT_event_extraction.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取所有事件
events = [item["Event"] for item in data]

# 初始化模型
model = SentenceTransformer('all-MiniLM-L6-v2')

# 计算事件的句子嵌入
embeddings = model.encode(events, convert_to_tensor=True)

# 计算相似度矩阵
similarities = util.pytorch_cos_sim(embeddings, embeddings)

# 对相似度进行排序
sorted_indices = similarities.argsort(dim=-1, descending=True)

# 构建结果数据
sorted_events = []
for i, event in enumerate(events):
    similar_events = [{"Similar_Event": events[j], "Score": similarities[i][j].item()} 
                      for j in sorted_indices[i] if i != j]
    sorted_events.append({"Event": event, "Similar_Events": similar_events})

# 保存结果到新的JSON文件
with open('sorted_events.json', 'w', encoding='utf-8') as f:
    json.dump(sorted_events, f, ensure_ascii=False, indent=4)

print("Results saved to sorted_events.json")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Results saved to sorted_events.json


In [6]:
import json

# 加载JSON文件
with open('sorted_events.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 处理每个元素
processed_data = []
for item in data:
    # 只保留前5个相似事件
    top_5_similar_events = sorted(item["Similar_Events"], key=lambda x: x["Score"], reverse=True)[:5]
    processed_item = {
        "Event": item["Event"],
        "Similar_Events": top_5_similar_events
    }
    processed_data.append(processed_item)

# 分割并保存每100个元素到一个JSON文件
for i in range(0, len(processed_data), 100):
    chunk = processed_data[i:i + 100]
    file_name = f'processed_data_part_{i//100 + 1}.json'
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(chunk, f, ensure_ascii=False, indent=4)

print("Processing and splitting complete. Files saved.")

Processing and splitting complete. Files saved.


写一个python程序，给当前json文件中每个element添加一个ID，从00001开始，下面是json中的一个element

In [2]:
import json

# 读取JSON文件
with open('/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/GAS_SPAS_EVENT_event_extraction.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 为每个元素添加唯一的ID
for i, element in enumerate(data, start=1):
    element['ID'] = f"{i:05d}"

# 将更新后的数据写回JSON文件
with open('/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/GAS_SPAS_EVENT_event_extraction_.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("IDs have been added successfully.")

IDs have been added successfully.


In [4]:
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# 加载JSON文件
with open('/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/GAS_SPAS_EVENT_event_extraction_.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取所有事件和对应的ID
events = [item["Event"] for item in data]
ids = [item["ID"] for item in data]

# 初始化模型
model = SentenceTransformer('all-MiniLM-L6-v2')

# 计算事件的句子嵌入
embeddings = model.encode(events, convert_to_tensor=True)

# 计算相似度矩阵
similarities = util.pytorch_cos_sim(embeddings, embeddings)

# 设置相似度阈值
threshold = 0.55

# 聚类
clusters = []
visited = set()

for i, event in tqdm(enumerate(events)):
    if i in visited:
        continue
    # 创建新集群
    cluster = {"Cluster_Events": [], "IDs": []}
    for j in range(len(events)):
        if similarities[i][j].item() > threshold:
            cluster["Cluster_Events"].append(events[j])
            cluster["IDs"].append(ids[j])
            visited.add(j)
    clusters.append(cluster)

# 保存结果到新的JSON文件
with open('clustered_events.json', 'w', encoding='utf-8') as f:
    json.dump(clusters, f, ensure_ascii=False, indent=4)

print("Clustering complete. Results saved to clustered_events.json")


/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
1852it [00:14, 131.27it/s]

Clustering complete. Results saved to clustered_events.json


In [5]:
import json
from sentence_transformers import SentenceTransformer, util
from collections import defaultdict

# 加载JSON文件
with open('GAS_SPAS_EVENT_event_extraction_.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取事件和ID
events = [(item["ID"], item["Event"]) for item in data]

# 初始化模型
model = SentenceTransformer('all-MiniLM-L6-v2')

# 计算事件的句子嵌入
embeddings = model.encode([event[1] for event in events], convert_to_tensor=True)

# 计算相似度矩阵
similarities = util.pytorch_cos_sim(embeddings, embeddings)

# 聚类相似事件
clusters = defaultdict(list)
visited = set()

# 设置相似度阈值
threshold = 0.6

for i in range(len(events)):
    if i in visited:
        continue
    cluster = []
    for j in range(len(events)):
        if i != j and similarities[i][j] > threshold:
            cluster.append({
                "ID": events[j][0],
                "Event": events[j][1],
                "Similarity_Score": similarities[i][j].item()
            })
            visited.add(j)
    if cluster:
        # cluster.append({
        #     "ID": events[i][0],
        #     "Event": events[i][1],
        #     "Similarity_Score": 1.0  # 自身与自身的相似度为1
        # })
        clusters[f"Cluster_{i}"] = cluster
    visited.add(i)

# 保存结果到新的JSON文件
with open('clustered_events_.json', 'w', encoding='utf-8') as f:
    json.dump(clusters, f, ensure_ascii=False, indent=4)

print("Clustering complete. Results saved to clustered_events.json")


Clustering complete. Results saved to clustered_events.json


写一个python程序，只保留similarity 0.6以上的，json文件中的其中一个element如下

In [6]:
import json

# 加载JSON文件
with open('/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/clustered_events_0_55.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 过滤相似度小于0.6的元素
filtered_data = {}
for cluster_id, events in data.items():
    filtered_events = [event for event in events if event["Similarity_Score"] >= 0.6]
    if filtered_events:  # 如果过滤后仍有事件，才保留该群组
        filtered_data[cluster_id] = filtered_events

# 保存结果到新的JSON文件
with open('clustered_events_0_6.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print("Filtering complete. Results saved to filtered_clustered_events.json")


Filtering complete. Results saved to filtered_clustered_events.json


下面是一个Python程序，它可以遍历JSON文件，并将每个element中最后一个 Similarity_Score 为 1.0 的项目删除：

In [1]:
import json

def remove_last_similarity_score(json_data):
    # 遍历所有的cluster
    for cluster_key, events in json_data.items():
        # 找到最后一个 similarity_score == 1.0 的索引
        index_to_remove = None
        for i, event in enumerate(events):
            if event.get("Similarity_Score") == 1.0:
                index_to_remove = i

        # 如果找到了这样的event，删除它
        if index_to_remove is not None:
            del json_data[cluster_key][index_to_remove]

    return json_data

# 读取json文件
with open('/home/qiang/projects/Digital_mental_health/Dataset/Event_Extraction/clustered_events_0_6.json', 'r') as file:
    data = json.load(file)

# 调用函数处理数据
updated_data = remove_last_similarity_score(data)

# 将结果写入新的json文件
with open('clustered_events_0_6_final.json', 'w') as file:
    json.dump(updated_data, file, indent=4)

print("处理完成，结果已保存到output.json")


处理完成，结果已保存到output.json


根据重复的ID，进行去重

In [2]:
import json

def extract_ids_from_json1(json1_data):
    """从第一个JSON文件中提取所有的ID"""
    ids = set()
    for cluster_key, events in json1_data.items():
        for event in events:
            ids.add(event.get("ID"))
    return ids

def remove_elements_by_id(json2_data, ids_to_remove):
    """从第二个JSON文件中删除包含指定ID的元素"""
    filtered_json2_data = [item for item in json2_data if item.get("ID") not in ids_to_remove]
    return filtered_json2_data

# 读取第一个JSON文件
with open('clustered_events_0_6_final.json', 'r') as file1:
    json1_data = json.load(file1)

# 读取第二个JSON文件
with open('GAS_SPAS_EVENT_event_extraction_.json', 'r') as file2:
    json2_data = json.load(file2)

# 提取第一个JSON文件中的所有ID
ids_to_remove = extract_ids_from_json1(json1_data)

# 删除第二个JSON文件中包含这些ID的元素
updated_json2_data = remove_elements_by_id(json2_data, ids_to_remove)

# 将结果写入新的JSON文件
with open('GAS_SPAS_EVENT_event_extraction_deduplication.json', 'w') as file:
    json.dump(updated_json2_data, file, indent=4)

print("处理完成，结果已保存到 deduplication.json")


处理完成，结果已保存到 deduplication.json
